In [ ]:
# Basic setup
!pip install transformers accelerate bitsandbytes datasets
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q faiss-cpu  # Optional if you want to use retrieval

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import random
import numpy as np


In [ ]:
model_name = "gpt2"  # You can later switch to LLaMA or Mistral

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
session_contexts = [
    "User: What's the weather like today?\nAI:",
    "User: Book me a cab to the airport.\nAI:",
    "User: Can you recommend a vegetarian recipe?\nAI:",
]

# Simulate user personalization tokens
user_profile = {"user_id": 42, "preferences": ["vegetarian", "travel", "weather"]}


In [ ]:
class KVCacheSimulator:
    def __init__(self):
        self.cache = []

    def add(self, token_id, importance=1.0):
        self.cache.append({"token": token_id, "importance": importance})

    def evict_fifo(self):
        if self.cache:
            self.cache.pop(0)

    def evict_least_important(self):
        self.cache.sort(key=lambda x: x["importance"])
        self.cache.pop(0)

    def print_cache(self):
        print([token["token"] for token in self.cache])


In [ ]:
def sage_kv_attention_sparsity(attn_weights):
    return attn_weights.mean(dim=-1).tolist()  # Simplified

def ke_diff_similarity(kv_cache):
    similarities = []
    for i in range(1, len(kv_cache)):
        sim = np.dot(kv_cache[i]["token"], kv_cache[i-1]["token"])
        similarities.append(sim)
    return similarities


In [ ]:
input_text = session_contexts[2]
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=30, return_dict_in_generate=True, output_scores=True)
generated = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

print("Generated:", generated)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: User: Can you recommend a vegetarian recipe?
AI: I've tried a lot of vegetarian recipes, but I've never had a problem with them. I've tried a lot of vegetarian recipes, but I


In [ ]:
import pandas as pd


In [ ]:
# Example cache state
cache = [
    {"token": "User", "importance": 0.9},
    {"token": "vegetarian", "importance": 0.8},
    {"token": "airport", "importance": 0.6},
    {"token": "recipe", "importance": 0.95},
]

# Create DataFrame
df_cache = pd.DataFrame(cache)
display(df_cache)


,token,importance
0,User,0.90
1,vegetarian,0.80
2,airport,0.60
3,recipe,0.95


In [ ]:
class KVCacheSimulatorPandas:
    def __init__(self):
        self.df = pd.DataFrame(columns=["token", "importance"])

    def add(self, token, importance):
        new_row = pd.DataFrame([{"token": token, "importance": importance}])
        self.df = pd.concat([self.df, new_row], ignore_index=True)

    def evict_fifo(self):
        self.df = self.df.iloc[1:].reset_index(drop=True)

    def evict_least_important(self):
        idx = self.df["importance"].idxmin()
        self.df = self.df.drop(index=idx).reset_index(drop=True)

    def show_cache(self):
        display(self.df)


In [ ]:
cache_sim = KVCacheSimulatorPandas()

# Adding tokens
cache_sim.add("User", 0.9)
cache_sim.add("vegetarian", 0.8)
cache_sim.add("airport", 0.6)
cache_sim.add("recipe", 0.95)

# Showing current cache
cache_sim.show_cache()

# Evicting least important
cache_sim.evict_least_important()
cache_sim.show_cache()


<ipython-input-14-7e0320cf3e23>:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, new_row], ignore_index=True)


,token,importance
0,User,0.90
1,vegetarian,0.80
2,airport,0.60
3,recipe,0.95


,token,importance
0,User,0.90
1,vegetarian,0.80
2,recipe,0.95


from matplotlib import pyplot as plt
_df_0['importance'].plot(kind='hist', bins=20, title='importance')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('token').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['importance'].plot(kind='line', figsize=(8, 4), title='importance')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_3['token'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_3, x='importance', y='token', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
eviction_log = []

def log_eviction(token, reason):
    eviction_log.append({"token": token, "reason": reason})

# Log example
log_eviction("airport", "least_important")

# Convert to DataFrame
df_evictions = pd.DataFrame(eviction_log)
display(df_evictions)


,token,reason
0,airport,least_important


# 🧠 Personalization-Aware KV Cache Eviction
This notebook explores techniques for prioritizing and evicting token-level key-value (KV) cache entries in multi-turn LLM inference, with a focus on **user-specific behavior** and **attention-based importance**.

The goal is to eventually train a lightweight model that can learn to decide which tokens to retain or evict during long conversations.



In [ ]:
!pip install transformers datasets pandas matplotlib --quiet

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2", output_attentions=True)
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
session = [
    "Hello, I need help with my math homework.",
    "Sure! What topic are you struggling with?",
    "It's about solving quadratic equations.",
    "Great. Can you give me one example?"
]


In [ ]:
token_data = []

for turn_id, sentence in enumerate(session):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        output = model(**inputs)

    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    attn_scores = output.attentions[-1][0][-1].cpu().numpy()  # attention from last layer, last head

    for i, token in enumerate(tokens):
        token_data.append({
            "turn": turn_id,
            "token": token,
            "token_id": int(inputs['input_ids'][0][i]),
            "attention": float(attn_scores[i]),
        })


`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
token_data = []

for turn_id, sentence in enumerate(session):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        output = model(**inputs)

    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # ---- attention handling ----
    attn = output.attentions[-1][0]                 # (heads, q_len, k_len)
    attn_scores = attn.mean(dim=0).mean(dim=0)      # (k_len,)
    attn_scores = attn_scores.cpu().numpy()

    for i, token in enumerate(tokens):
        token_data.append({
            "turn":       turn_id,
            "token":      token,
            "token_id":   int(inputs['input_ids'][0][i]),
            "attention":  float(attn_scores[i]),
        })


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd

# 1. Load model
model = AutoModelForCausalLM.from_pretrained("distilgpt2", output_attentions=True, attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model.eval()

# 2. Session input
session = [
    "Hi, how are you?",
    "Can you suggest me a good movie?",
    "Thanks!"
]

# 3. Get token-level attention
token_data = []

for turn_id, sentence in enumerate(session):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        output = model(**inputs)

    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    attn = output.attentions[-1][0]                # (heads, q_len, k_len)
    attn_scores = attn.mean(dim=0).mean(dim=0)     # (k_len,)
    attn_scores = attn_scores.cpu().numpy()

    for i, token in enumerate(tokens):
        token_data.append({
            "turn": turn_id,
            "token": token,
            "token_id": int(inputs['input_ids'][0][i]),
            "attention": float(attn_scores[i]),
        })

# 4. Display as table
df = pd.DataFrame(token_data)
display(df.head(20))


The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


,turn,token,token_id,attention
0,0,Hi,17250,0.782039
1,0,",",11,0.107942
2,0,Ġhow,703,0.048671
3,0,Ġare,389,0.028860
4,0,Ġyou,345,0.011487
5,0,?,30,0.021002
6,1,Can,6090,0.775465
7,1,Ġyou,345,0.073591
8,1,Ġsuggest,1950,0.058801
9,1,Ġme,502,0.031654


In [ ]:
df['evict'] = df['attention'] < 0.05  # Example threshold



In [ ]:
X = df[['token_id', 'turn', 'attention']]
y = df['evict']


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)


LogisticRegression()

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)


LogisticRegression()

In [ ]:
from sklearn.metrics import accuracy_score

# Predict on the same data (just for a basic check)
y_pred = model.predict(X)

# See accuracy
print("Training Accuracy:", accuracy_score(y, y_pred))


Training Accuracy: 0.6875


In [ ]:
df['predicted_evict'] = y_pred
df[['token', 'attention', 'evict', 'predicted_evict']].head(10)


,token,attention,evict,predicted_evict
0,Hi,0.782039,False,False
1,",",0.107942,False,True
2,Ġhow,0.048671,True,True
3,Ġare,0.028860,True,True
4,Ġyou,0.011487,True,True
5,?,0.021002,True,True
6,Can,0.775465,False,False
7,Ġyou,0.073591,False,True
8,Ġsuggest,0.058801,False,True
9,Ġme,0.031654,True,True


In [ ]:
import torch.nn as nn

class TokenImportanceModel(nn.Module):
    def __init__(self, input_dim):
        super(TokenImportanceModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()  # Output score between 0 and 1

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.sigmoid(self.fc3(x))


***From here starting the tagging system, the main idea***

In [ ]:
import pandas as pd

# Sample prompts and topic labels
data = {
    "prompt": [
        "How does retrieval-augmented generation work?",
        "Can I eat three chilas at once?",
        "How to improve aim in Valorant?",
        "Tell me about KV cache in LLMs",
        "Give me a diet plan for weight loss",
        "What is the best crosshair for Vandal?",
        "Can you help me with my gym routine?",
        "Difference between BERT and GPT?",
        "Suggest me good graphics settings for Valorant",
        "Will skipping dinner help in fat loss?"
    ],
    "label": [
        "LLM",
        "Weight loss",
        "Gaming",
        "LLM",
        "Weight loss",
        "Gaming",
        "Weight loss",
        "LLM",
        "Gaming",
        "Weight loss"
    ]
}

df = pd.DataFrame(data)
df



,prompt,label
0,How does retrieval-augmented generation work?,LLM
1,Can I eat three chilas at once?,Weight loss
2,How to improve aim in Valorant?,Gaming
3,Tell me about KV cache in LLMs,LLM
4,Give me a diet plan for weight loss,Weight loss
5,What is the best crosshair for Vandal?,Gaming
6,Can you help me with my gym routine?,Weight loss
7,Difference between BERT and GPT?,LLM
8,Suggest me good graphics settings for Valorant,Gaming
9,Will skipping dinner help in fat loss?,Weight loss


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Encode the labels (text to integers)
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# TF-IDF vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['prompt'])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, df['label_encoded'], test_size=0.2, random_state=42)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Predict
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# --- Fix: only pass the labels that actually appear in y_test ---
unique_test_labels = np.unique(y_test)
print(
    "\nClassification Report:\n",
    classification_report(
        y_test,
        y_pred,
        labels=unique_test_labels,
        target_names=label_encoder.inverse_transform(unique_test_labels),
        zero_division=0,   # avoids divide‑by‑zero warnings for small datasets
    ),
)


Accuracy: 0.50

Classification Report:
               precision    recall  f1-score   support

      Gaming       0.00      0.00      0.00         1
 Weight loss       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



In [ ]:
def classify_prompt(prompt):
    vec = vectorizer.transform([prompt])
    pred = model.predict(vec)
    return label_encoder.inverse_transform(pred)[0]

# Test
print(classify_prompt("Help me with fine-tuning GPT models"))
print(classify_prompt("Can I have cold coffee on keto?"))
print(classify_prompt("How to rank up in Valorant?"))


Weight loss
Weight loss
Gaming


In [ ]:
llm_prompts = [
    "What are key-value caches in transformers?",
    "Explain retrieval-augmented generation with an example.",
    "How can LLMs adapt to user behavior over time?",
    "What is SAGE-KV in attention mechanisms?",
    "How does a sliding window attention work in transformers?",
    "Which models support dynamic context expansion?",
    "Compare rotary positional embeddings and absolute embeddings.",
    "What does attention sparsity mean in transformer models?",
    "How does fine-tuning differ from instruction tuning?",
    "Can BART be used with FAISS for retrieval?",
    "Explain token prioritization in multi-turn LLMs.",
    "Are KV caches useful in inference optimization?",
    "What is the benefit of dropout in transformers?",
    "How do LoRA adapters modify transformer layers?",
    "What’s the best cache eviction strategy for long sessions?",
    "What is dynamic prompt injection?",
    "How can user personalization improve chatbot performance?",
    "What is the role of embeddings in document retrieval?",
    "How does FAISS work under the hood?",
    "What are sentence embeddings used for?",
    "Explain BERT vs SBERT in simple terms.",
    "Which LLMs support streaming token output?",
    "What are attention gates used for in LLMs?",
    "How do we quantify token importance in a KV cache?",
    "What is token salience ranking?",
    "Is KV cache storage a bottleneck in LLMs?",
    "Explain the concept of token-level importance modeling.",
    "How does OpenAI's GPT handle multi-user memory?",
    "What are the risks of incorrect KV cache eviction?",
    "Which metrics are used to evaluate cache-aware LLMs?"
]


In [ ]:
weight_loss_prompts = [
    "Can I eat 2 chillas at once during diet?",
    "Is cold coffee okay for weight loss?",
    "What can I replace jaggery with in tea?",
    "How many calories in paneer bhurji?",
    "Can I do intermittent fasting with Indian food?",
    "Is roti better than rice for weight loss?",
    "What time should I stop eating at night?",
    "How to plan a vegetarian low-carb diet?",
    "Can I drink fruit juice while dieting?",
    "Is ghee harmful during weight loss?",
    "Is skipping dinner better than breakfast?",
    "How often should I eat during IF?",
    "What is a good weight loss chutney recipe?",
    "Can I use rock salt in weight loss meals?",
    "Is poha a good breakfast for dieting?",
    "Can I drink buttermilk while fasting?",
    "What are slow carbs and fast carbs?",
    "Is it okay to have cheat meals?",
    "How many steps should I walk per day?",
    "Can I drink flavored water during fast?",
    "What’s a good low-calorie Indian dessert?",
    "How to reduce bloating during diet?",
    "Can I eat after 8 PM during weight loss?",
    "Is coconut water good for weight loss?",
    "Can I use peanut butter in my diet?",
    "Should I stop dairy to lose weight?",
    "What foods cause water retention?",
    "Best home workouts without equipment?",
    "Can I add lemon to my green tea?",
    "Does fasting affect metabolism long-term?"
]


In [ ]:
valorant_prompts = [
    "How to fix low FPS in Valorant?",
    "Why is Valorant stuck at 30 FPS?",
    "What’s the best sensitivity for beginners?",
    "How to check GPU usage while gaming?",
    "Should I use low latency mode in NVIDIA settings?",
    "How to enable high performance for Valorant?",
    "Which is the best agent for ranked solo play?",
    "How to reduce input lag in Valorant?",
    "Best crosshair settings for headshots?",
    "How to use killjoy effectively?",
    "Can I play Valorant with integrated graphics?",
    "How to set Valorant on dedicated GPU?",
    "What are the best system settings for smoother gameplay?",
    "How to update drivers for Valorant performance?",
    "Should I cap FPS or leave it unlocked?",
    "Why does my ping spike in Valorant?",
    "Can Discord cause FPS drops in Valorant?",
    "What does 'anti-aliasing' do in Valorant settings?",
    "Best resolution for competitive Valorant?",
    "What is 4:3 stretched resolution and is it allowed?",
    "How to prioritize RAM for gaming?",
    "Is Valorant CPU or GPU heavy?",
    "What is the best monitor refresh rate for Valorant?",
    "How to check if Valorant is using dedicated GPU?",
    "Does RAM size affect FPS in Valorant?",
    "How to fix Valorant update errors?",
    "Can I undervolt CPU for better temps while gaming?",
    "Does Valorant support dual monitors?",
    "What is the ideal aim training routine?",
    "How to record Valorant gameplay without lag?"
]


In [ ]:
import pandas as pd

# Combine prompts and labels into a DataFrame
data = pd.DataFrame({
    "prompt": llm_prompts + weight_loss_prompts + valorant_prompts,
    "label": ["llm"]*30 + ["weight_loss"]*30 + ["valorant"]*30
})

# Shuffle for good measure
data = data.sample(frac=1).reset_index(drop=True)

# Preview
data.head()


,prompt,label
0,Which is the best agent for ranked solo play?,valorant
1,How to record Valorant gameplay without lag?,valorant
2,How to fix low FPS in Valorant?,valorant
3,What is 4:3 stretched resolution and is it all...,valorant
4,What are the risks of incorrect KV cache evict...,llm


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label'])

# Print mapping
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


{'llm': np.int64(0), 'valorant': np.int64(1), 'weight_loss': np.int64(2)}


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['prompt'])
y = data['label_encoded']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=2000)
clf.fit(X_train, y_train)


LogisticRegression(max_iter=2000)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.94
              precision    recall  f1-score   support

         llm       1.00      1.00      1.00         7
    valorant       0.88      1.00      0.93         7
 weight_loss       1.00      0.75      0.86         4

    accuracy                           0.94        18
   macro avg       0.96      0.92      0.93        18
weighted avg       0.95      0.94      0.94        18



In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X, y, cv=5)
print("Cross-validation scores:", scores)
print("Average:", scores.mean())


Cross-validation scores: [0.94444444 0.83333333 0.66666667 0.88888889 1.        ]
Average: 0.8666666666666666
